![img](https://raw.githubusercontent.com/rohan-varma/paper-analysis/master/word2vec-papers/models.png)

### Introduction

The Word2Vec model has become a standard method for representing words as dense vectors. This is typically done as a preprocessing step, after which the learned vectors are fed into a discriminative model (typically an RNN) to generate predictions such as movie review sentiment, do machine translation, or even generate text, character by character (TODO link to char-rnn). 

### Previous Language Models

Previously, the bag of words model was commonly used to represent words and sentences as numerical vectors, which could then be fed into a classifier (for example Naive Bayes) to produce output predictions. Given a vocabulary of $V$ words and a document of $N$ words, a $V$-dimensional vector woudl be created to represent the vector, where index $i$ denotes the number of times the $i$th word in the vocabulary occured in the document. 

This model represented words as atomic units, assuming that all words were independent of each other. It had success in several fields such as document classification, spam detection, and even sentiment analysis, but its assumptions (that words are completely independent of each other) were too strong for more powerful and accurate models. A model that aimed to reduce some of the strong assumptions of the traditional bag of words model was the n-gram model. 

### N-gram models and Markov Chains

Language models seek to predict the probability of the $t + 1$th word $w_{t + 1}$ given the previous $t$ words: 

$$p(w_{t + 1} | w_1, w_2, ... w_t)$$

Using the chain rule of probabilty, we can compute the probabilty of observing an entire sentence: 

$$p(w_1, w_2, ... w_t) = p(w_1)p(w_2 | w_1)...p(w_t | w_{t -1}, ... w_1)$$

Computing these probabilities have many applications, for example in speech recognition, spelling corrections, and automatic sentence completion. However, estimating these probabilites can be tough. We can use the maximum likelihood estimate: 

$$p(x_{t + 1} | x_1, ... x_t) = \frac{count(x_1, x_2, ... x_t, x_{t + 1})}{count(x_1, x_2, ... x_t)}$$

However, computing this is quite unrealistic - we will generally not observe enough data from a corpus to obtain realistic counts for any sequence of $t$ words for any nontrivial value of $t$, so we instead invoke the Markov assumption. The Markov assumption assumes that the probability of observing a word at a given time is only dependent on the word observed in the previous time step, and independent of the words observed in all of the previous time steps: 

$$p(x_{t + 1} | x_1, x_2, ... x_t) = p(x_{t + 1} | x_t) $$

Therefore, the probabilty of a sentence can be given by 

$$p(w_1, w_2, ... w_t) = p(w_1)\prod_{i = 2}^{t} p(w_i | w_{i - 1})$$

The Markov assumption can be extended to condition the probability of the $t$th word on the previous two, three, four, and so on words. This is where the name of the n-gram model comes in - n is the number of previous timesteps we condition the current timestep on. Some examples:

Unigram Model: $p(x_{t + 1} | x_1, x_2, ... x_t) = p(x_{t + 1})$

Bigram Model: $p(x_{t + 1} | x_1, x_2, ... x_t) = p(x_{t + 1} | x_t)$

There is a lot more to the n-gram model such as linear interpolation and smoothing techniques, which [these slides](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf) explain very well. 

### The Skip-Gram and Continuous Bag of Words Models

Word vectors, or word embeddings, or distributed representation of words, generally refer to a dense vector representation of a word, as compared to a sparse (ie one-hot) traditional representation. There are actually two different implementations of models that learn dense representation of words: the Skip-Gram model and the Continuous Bag of Words model. Both of these models learn dense vector representation of words, based on the words that surround them (ie, their *context*). 

The difference is that the skip-gram model predicts context (surrounding) words given the current word, wheras the continuous bag of words model predicts the current word based on several surrounding words. 

This notion of "surrounding" words is best described by considering a center (or current) word and a window of words around it. For example, if we consider the sentence "The quick brown fox jumped over the lazy dog", and a window size of 2, we'd have the following pairs for the skip-gram model:

![img](http://mccormickml.com/assets/word2vec/training_data.png)

In contrast, for the CBOW model, we'll input the context words within the window (such as "the", "brown", "fox") and aim to predict the target word "quick" (simply reversing the input -> prediction pipeline from the skip-gram model). 

The following is a visualization of the skip-gram and CBOW models:

![img](https://raw.githubusercontent.com/rohan-varma/paper-analysis/master/word2vec-papers/models.png)

In this [paper](https://arxiv.org/pdf/1301.3781.pdf), the overall recommendation was to use the skip-gram model, since it had been shown to perform better on analogy-related tasks than the CBOW model. Overall, if you understand one model, it is pretty easy to understand the other: just reverse the inputs and predictions. Since both papers focused on the skip-gram model, this post will do the same. 

### Learning with the Skip-Gram Model

Our goal is to find word representations that are useful for predicting the surrounding words given a current word. 
In particular, we wish to maximize the average log probability across our entire corpus: 

$$ argmax_{\theta} \frac{1}{T} \sum_{t=1}^{T} \sum_{j \in c, j != 0} log  p(w_{t + j} | w_{t} ; \theta) $$

This equation essentially says that there is some probability $p$ of observing a particular word that's within a window of size $c$ of the current word $w_t$. This probability is conditioned on the current word ($w_t$) and some setting of parameters $\theta$ (determined by our model). We wish to set these parameters $\theta$ so that this probability is maximized across our entire corpus.

### Basic Parametrization: Softmax Model

The basic skip-gram model defines $p(w_{t + j} | w_t)$ by the softmax function. If we consider $w_{t + j}$ to be a one-hot encoded vector with dimension $N$ and $\theta$ to be a $N * K$ matrix embedding matrix (here, we have $N$ words in our vocabulary and our learned embeddings have dimension $K$), then we can define $p(w_{t + j} | w_t ; \theta) = \frac{exp(\theta w_{t + j})}{\sum_t exp(\theta w_t)}$. It is worth noting that after learning, the matrix $\theta$ can be thought of as an embedding lookup matrix. If you have a word that is represented with the $k$th index of a vector being hot, then the learning embedding for that word will be the $k$th column. 

This parametrization has a major disadvantage that limits its usefulness in cases of very large corpuses. Specifically, we notice that in order to compute a single forward pass of our model, we must sum across the entire corpus vocabulary in order to evaluate the softmax function. This is prohibitively expensive on large datasets, so we look to alternate approximations of this model for the sake of computational efficiency. 


### Hierarchical Softmax and Negative Sampling

As discussed, the traditional softmax approach can become prohibitively expensive on large corpora, and the hierarchical softmax is a common alternative approach that approximates the softmax computation, but has logarithmic time complexity in the number of words in the vocabulary, as opposed to linear time complexity. 

This is done by representing the softmax layer as a binary tree where the words are leaf nodes of the tree, and the probabilities are computed by a walk from the root of the binary tree to the particular leaf. 






### Sources

[Link](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) to paper. 

Some insights from [this paper](https://arxiv.org/pdf/1301.3781.pdf) were also used.
